In [1]:
# Import necessary packages
import torch
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
torch.cuda.device_count()  # print 1
import torchdeepretina as tdr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchdeepretina.io as tdrio
import torchdeepretina.utils as tdrutils
import stimuli as s
import torchdeepretina.stimuli as tdrstim
from tqdm import tqdm
from itertools import repeat
from matplotlib import ticker, cm
from colormap import Colormap
c = Colormap()
mycmap = c.cmap_linear('#2378FF','#FFFFFF','#FF3C3C')

f2_response is unavailable until you run:
$ pip install -e git+git://github.com/nirum/jetpack.git@master#egg=jetpack


In [2]:
device = torch.device("cuda:0")
# load model given path
def load_model(model_path):
    """
    in: model_path
    out: model
    """
    model = tdrio.load_model(model_path)
    model.to(device)
    model.eval()
    return model

model_dir_path = '/home/htanaka/torch-deep-retina/models_paper/'#/tanaka/convgc_models/gcchansearch_57_chans[4, 4]/'

#model_n_path = 'convgc_15-10-07_whitenoise'

#model_n_path = '15-11-21b_naturalscene'
#model_n_path = '15-10-07_naturalscene'
#model_n_path = 'convgc_15-10-07_naturalscene'

model_n_path = 'convgc_15-11-21b_naturalscene'
model_n = load_model(model_dir_path + model_n_path + '.pt')
model_n = tdr.utils.stacked2conv(model_n)

model_w_path = 'convgc_15-11-21b_whitenoise'
model_w = load_model(model_dir_path + model_w_path + '.pt')
model_w = tdr.utils.stacked2conv(model_w)

In [4]:
# Motion anticipation
def motion_anticipation(velocity=0.08, width=6, flash_duration=2, filt_depth=40):
    c_right, speed_right, stim_right = tdrstim.driftingbar(velocity,    
                                                 width, x=(-30, 30))    
    rightward_moving_bar = torch.from_numpy(stim_right)
    c_left, speed_left, stim_left = tdrstim.driftingbar(-velocity,      
                                               width, x=(30, -30))      
    leftward_moving_bar = torch.from_numpy(stim_left)
    flash_centers = np.arange(-25, 26)
    flashes = (tdrstim.flash(flash_duration, 43, 70,                    
                        intensity=tdrstim.bar((x, 0), width, 50)) for x in flash_centers)
    xs = []
    for f in flashes:
        x = torch.from_numpy(tdrstim.concat(f, nh=filt_depth))
        xs.append(x)
    return xs, rightward_moving_bar, c_right, leftward_moving_bar, c_left




# Motion reversal
def run_motion_reversal(x_locations=np.arange(-9, 3), speed=0.19, clip_n=210, scaling=1):
    """Gets responses to a bar reversing motion."""                        
    tflips, Xs = zip(*[s.motion_reversal(xi,speed)[1:] for xi in x_locations])
    return tflips, Xs

In [6]:
# Motion anticipation
def motion_anticipation(velocity=0.08, width=6, flash_duration=2, filt_depth=40):
    # Generate rightward moving bar stimulus
    c_right, speed_right, stim_right = tdrstim.driftingbar(velocity, width, x=(-30, 30))
    rightward_moving_bar = torch.from_numpy(stim_right)
    
    # Generate leftward moving bar stimulus
    c_left, speed_left, stim_left = tdrstim.driftingbar(-velocity, width, x=(30, -30))
    leftward_moving_bar = torch.from_numpy(stim_left)
    
    # Generate flashes at different positions
    flash_centers = np.arange(-25, 26)
    flashes = [tdrstim.flash(flash_duration, 43, 70, intensity=tdrstim.bar((x, 0), width, 50)) for x in flash_centers]
    
    # Convert flashes to PyTorch tensors and concatenate
    xs = [torch.from_numpy(tdrstim.concat(f, nh=filt_depth)) for f in flashes]
    
    # Return stimuli and velocities
    return xs, rightward_moving_bar, c_right, leftward_moving_bar, c_left

# Motion reversal
def run_motion_reversal(x_locations=np.arange(-9, 3), speed=0.19, clip_n=210, scaling=1):
    """Gets responses to a bar reversing motion."""                        
    tflips, Xs = zip(*[s.motion_reversal(xi,speed)[1:] for xi in x_locations])
    return tflips, Xs


In [11]:
from tqdm import tqdm

# Generate motion anticipation stimuli
flash, rightward_moving_bar, c_right, leftward_moving_bar, c_left = motion_anticipation(velocity=0.08, width=6, flash_duration=2, filt_depth=40)

# Loop over channels
for i in tqdm(range(int(model_n.n_units))):
    # Compute integrated gradient and response for rightward moving bar
    integrad_r, response_r = tdrutils.integrated_gradient(model=model_n, X=rightward_moving_bar, layer="sequential.0", chans=i, alpha_steps=100)
    integrad_r, response_r = integrad_r.numpy(), response_r.numpy()
    
    # Save results
    np.save('./analysis_data/{}/motion_anticipation/integrad_r_chan_{}_steps_{}.npy'.format(model_n_path, i, integ_steps), integrad_r)
    np.save('./analysis_data/{}/motion_anticipation/response_r_chan_{}_steps_{}.npy'.format(model_n_path, i, integ_steps), response_r)
    
    # Compute integrated gradient and response for leftward moving bar
    integrad_l, response_l = tdrutils.integrated_gradient(model=model_n, X=leftward_moving_bar, layer="sequential.0", chans=i, alpha_steps=100)
    integrad_l, response_l = integrad_l.numpy(), response_l.numpy()
    
    # Save results
    #np.save('./analysis_data/{}/motion_anticipation/integrad_l_chan_{}_steps_{}.npy'.format(model_n_path, i, integ_steps), integrad_l)
    #np.save('./analysis_data/{}/motion_anticipation/response_l_chan_{}_steps_{}.npy'.format(model_n_path, i, integ_steps), response_l)


  0%|          | 0/17 [00:00<?, ?it/s]

NameError: name 'integ_steps' is not defined